In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ1: Which technologies have been investigated in the last decade?

In [3]:
#Datene einlesen
#data = pd.read_csv('./data/data.csv')
#data.drop(["Unnamed: 0"], axis = 1, inplace = True)

data = pd.read_csv('./data/export1.csv',dtype={'Abstract': str}, usecols = ['Document Title','Abstract','Publication Year'])
data.head()

,Document Title,Publication Year,Abstract
0,What Do We Know about Knowledge Management? Pr...,2009,There have been many claims about knowledge ma...
1,Research on National and International Softwar...,2009,The article analyzed the basic life cycle mode...
2,Advancing Software Engineering Professional Ed...,2011,The paper mentions that a reference curriculum...
3,Putting Human Aspects of Software Engineering ...,2010,Although people-related issues are central fac...
4,Where's the Theory for Software Engineering?,2012,"Darwin's theory of natural selection, Maxwell'..."


In [4]:
data.columns

Index(['Document Title', 'Publication Year', 'Abstract'], dtype='object')

In [5]:
data.count()

Document Title      1946
Publication Year    1946
Abstract            1933
dtype: int64

In [6]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2),dtype=np.int32)
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names(), dtype=np.int32)
    df['PublishingYear'] = data['Publication Year']
    returndf = df.groupby('PublishingYear').sum().transpose().astype('int32')
    return returndf

In [7]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [8]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

In [9]:
#Untersuchung Abstract
#abstract = CreateTermDocMatrix('Abstract')

In [10]:
#b = abstract.loc[abstract[2011] > 20] 

In [11]:
#b.plot.bar()
#print(b)

In [12]:
from collections import Counter
#cnt = Counter()

#for abstract in data['Abstract']:
#    if type(abstract) is str:
#        wordlist = abstract.split(' ')
#        for word in wordlist:
#            cnt[word] += 1
        
#cnt.most_common(30)

In [13]:
#for abstract in data.itertuples():
#    print(abstract[1])
#    print(abstract[2])
#    print(abstract[3])

In [14]:
data = pd.read_csv('./data/data.csv',usecols = ['Document Title','Abstract','Publication Year'])
data.columns = ['title','abstract','year']

In [15]:
#Untersuchung Title
#title = CreateTermDocMatrix('title')

#print(title)
#title = SpaltenEntf(title)
#print(title)

#b = title.loc[title[2017] > 5] 

#b.plot.bar()
#print(b)
#b.shape

In [16]:
#Einzelnes Wort anzeigen
#abstract = CreateTermDocMatrix('Abstract')
#ShowWord(abstract,'software')

In [17]:
import spacy
from spacy.symbols import NOUN
from collections import Counter


nlp = spacy.load("en_core_web_sm")

cnt = Counter()
tokenlist = []

for columns in data.itertuples():

    abstract = columns[1] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
    if type(abstract) is str:
        doc = nlp(abstract)
        
        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
            token_text = token.text
            token_pos = token.pos_
            token_dep = token.dep_
            token_ent = token.ent_type_

            if token.pos == NOUN:
                #print(f"{token_text}: {token_ent}")
                cnt[token_text] += 1
                tokenlist.append([token_text,columns[2],token_ent])


                # Dies dient nur zur Formatierung
        #        print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")
 
#cnt.most_common(50)
#tokenlist

[('Research', 112),
 ('Study', 98),
 ('cover', 77),
 ('Approach', 73),
 ('Web', 56),
 ('Implementation', 40),
 ('Design', 33),
 ('Case', 30),
 ('Performance', 30),
 ('Application', 30),
 ('Information', 29),
 ('Introduction', 28),
 ('Notice', 26),
 ('Prediction', 25),
 ('Challenges', 24),
 ('Editors', 22),
 ('Test', 22),
 ('Decision', 22),
 ('Evidence', 20),
 ('Networks', 20),
 ('Internet', 20),
 ('Time', 19),
 ('Testing', 19),
 ('Effect', 19),
 ('Projects', 19),
 ('authors', 18),
 ('Survey', 17),
 ('Requirements', 16),
 ('Bug', 16),
 ('Editorial', 16),
 ('Defect', 16),
 ('Theory', 15),
 ('Problems', 14),
 ('Retraction', 14),
 ('Component', 13),
 ('Analysis', 13),
 ('advertisement', 13),
 ('Execution', 13),
 ('Impact', 12),
 ('Evaluation', 12),
 ('Processing', 12),
 ('Modeling', 12),
 ('Cases', 12),
 ('Lessons', 12),
 ('Experiments', 11),
 ('End', 11),
 ('Features', 11),
 ('Years', 11),
 ('Self', 10),
 ('Lines', 10)]